In [31]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
import itertools
from sklearn.neighbors import KNeighborsRegressor
from sklearn.cluster import KMeans
import plotly.graph_objects as go
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from umap.umap_ import UMAP

C:\Users\ghibl\anaconda3\Lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning:

The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.

C:\Users\ghibl\anaconda3\Lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning:

The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.

C:\Users\ghibl\anaconda3\Lib\site-packages\umap\distances.py:1086: NumbaDeprecationWarning:

The 'nopython' keyword argument was not suppl

In [32]:
dataset_df = pd.read_csv(r"C:\Users\ghibl\ICR\data\input\train.csv")
X_df = dataset_df.drop("Id", axis = 1)

Do labelencoding for EJ column

In [33]:
le = LabelEncoder()
X_df["EJ"] = le.fit_transform(X_df["EJ"])

In [34]:
features_df = X_df.copy()

Relation between missing values and class in BQ and EL

In [35]:
miss_df = dataset_df[["BQ","EL","Class"]].copy()
miss_df["bq_na"] = miss_df["BQ"].isnull()
miss_df["el_na"] = miss_df["EL"].isnull()

bq_pivot = miss_df.pivot_table(index = "bq_na",columns = "Class", values = None,aggfunc = 'count')
el_pivot = miss_df.pivot_table(index = "el_na",columns = "Class", values = None,aggfunc = 'count')

print(bq_pivot)
print(el_pivot)

          BQ            EL         el_na       
Class      0      1      0      1      0      1
bq_na                                          
False  449.0  108.0  448.0  102.0  449.0  108.0
True     0.0    NaN    7.0    NaN   60.0    NaN
        BQ        EL      bq_na     
Class    0    1    0    1     0    1
el_na                               
False  448  102  455  102   455  102
True     1    6    0    0    54    6


complement missing values with median or average


In [36]:
null_columns =features_df.columns[features_df.isnull().any()]
null_columns = [i for i in null_columns if not i in ["BQ","EL"]]

for column in null_columns:
    median = features_df[column].median()
    features_df[column] = features_df[column].fillna(median)

# Predict missing values by KNN

BQ

In [37]:
#split the dataset to trainset and testset
bq_df = features_df.copy().drop(["Class","EL"],axis = 1)
null_index = bq_df[bq_df["BQ"].isnull()].index
train_bq = bq_df.drop(null_index,axis = 0)
test_bq = bq_df.iloc[null_index,:]

In [38]:
X_train_bq ,y_train_bq = train_bq.drop("BQ",axis = 1), train_bq["BQ"]
X_test_bq = test_bq.drop("BQ",axis = 1)

knn = KNeighborsRegressor(n_neighbors = 4)
knn.fit(X_train_bq,y_train_bq)
y_pred_bq = knn.predict(X_test_bq)

In [39]:
y_pred_bq = pd.Series(y_pred_bq,index = X_test_bq.index)
features_df["BQ"] = features_df["BQ"].fillna(y_pred_bq)

EL

In [40]:
el_df = features_df.copy().drop(["Class","BQ"],axis = 1)
null_index = el_df[el_df["EL"].isnull()].index
train_el = el_df.drop(null_index,axis = 0)
test_el = el_df.iloc[null_index,:]

In [41]:
X_train_el, y_train_el = train_el.drop("EL",axis = 1), train_el["EL"]
X_test_el = test_el.drop("EL",axis = 1)

knn = KNeighborsRegressor(n_neighbors = 4)
knn.fit(X_train_el,y_train_el)
y_pred_el = knn.predict(X_test_el)

In [42]:
y_pred_el = pd.Series(y_pred_el,index = X_test_el.index)
features_df["EL"] = features_df["EL"].fillna(y_pred_el)

In [43]:
print(features_df.shape)
features_df.isnull().sum()

(617, 57)


AB       0
AF       0
AH       0
AM       0
AR       0
AX       0
AY       0
AZ       0
BC       0
BD       0
BN       0
BP       0
BQ       0
BR       0
BZ       0
CB       0
CC       0
CD       0
CF       0
CH       0
CL       0
CR       0
CS       0
CU       0
CW       0
DA       0
DE       0
DF       0
DH       0
DI       0
DL       0
DN       0
DU       0
DV       0
DY       0
EB       0
EE       0
EG       0
EH       0
EJ       0
EL       0
EP       0
EU       0
FC       0
FD       0
FE       0
FI       0
FL       0
FR       0
FS       0
GB       0
GE       0
GF       0
GH       0
GI       0
GL       0
Class    0
dtype: int64

Save this no missing value dataframe into csv file

In [44]:
complemented_df = pd.concat([dataset_df["Id"],features_df],axis = 1)
complemented_df.to_csv(r'C:\Users\ghibl\ICR\data\input\processed_train.csv',index = False)

In [45]:
complemented_df.head()

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,...,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,...,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,...,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,...,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,...,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1


Standardize basic features

In [46]:
processed_df = features_df.copy()
num_cols = list(processed_df.columns)
num_cols = [i for i in num_cols if not i in ["EJ","Class"]]
sc = StandardScaler()
processed_df[num_cols] = sc.fit_transform(processed_df[num_cols])
processed_df_csv = pd.concat([dataset_df["Id"],processed_df],axis = 1)
processed_df_csv.to_csv(r'C:\Users\ghibl\ICR\data\input\processed_std_train.csv',index = False)

In [47]:
processed_df.drop("Class",axis = 1,inplace = True)

In [48]:
processed_df.isnull().sum()

AB     0
AF     0
AH     0
AM     0
AR     0
AX     0
AY     0
AZ     0
BC     0
BD     0
BN     0
BP     0
BQ     0
BR     0
BZ     0
CB     0
CC     0
CD     0
CF     0
CH     0
CL     0
CR     0
CS     0
CU     0
CW     0
DA     0
DE     0
DF     0
DH     0
DI     0
DL     0
DN     0
DU     0
DV     0
DY     0
EB     0
EE     0
EG     0
EH     0
EJ     0
EL     0
EP     0
EU     0
FC     0
FD     0
FE     0
FI     0
FL     0
FR     0
FS     0
GB     0
GE     0
GF     0
GH     0
GI     0
GL     0
dtype: int64

# PCA

In [49]:
pca = PCA(n_components=5)
pca_df = pd.DataFrame(pca.fit_transform(processed_df),
                      columns = ["pca_Component1","pca_Component2","pca_Component3","pca_Component4","pca_Component5"])

In [50]:
kmeans = KMeans(n_clusters = 2, random_state = 42)
kmeans.fit(pca_df)
features_clusters = kmeans.predict(pca_df)
pca_distances = kmeans.transform(pca_df)
pca_distances_df = pd.DataFrame(pca_distances, columns = ["pca_distance1","pca_distance2"])
pca_df["pca_Cluster"] = features_clusters
pca_df = pd.concat([pca_df,pca_distances_df],axis =1)

C:\Users\ghibl\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

C:\Users\ghibl\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



In [51]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = pca_df["pca_Component1"], y = pca_df["pca_Component2"],mode = "markers",
                        marker = dict(color = pca_df["pca_Cluster"])))

In [52]:
fig = go.Figure()
fig.add_trace(go.Scatter3d(x = pca_df["pca_Component1"], y = pca_df["pca_Component2"],z = pca_df["pca_Component3"],
                         mode = "markers",marker = dict(color = pca_df["pca_Cluster"])))

In [53]:
features_df = pd.concat([features_df,pca_df.drop("pca_Cluster",axis = 1)],axis = 1)

In [54]:
features_df

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,GI,GL,Class,pca_Component1,pca_Component2,pca_Component3,pca_Component4,pca_Component5,pca_distance1,pca_distance2
0,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,...,69.834944,0.120343,1,-1.738183,-1.109691,1.198878,0.864153,0.202589,2.496555,41.978253
1,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,...,32.131996,21.978000,0,-1.441889,-1.360409,-0.898552,-0.314330,0.676124,2.262222,41.708466
2,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,5135.78024,...,35.192676,0.196941,0,0.072946,0.434018,0.720496,1.319426,-1.221434,1.985197,41.057363
3,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,4169.67738,...,90.493248,0.155829,0,-0.359200,0.262762,0.684072,0.890236,-1.520192,1.917203,41.388206
4,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,5728.73412,...,36.262628,0.096614,1,-0.524386,0.788133,0.094595,1.569662,0.172711,1.793043,41.716341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,0.149555,3130.05946,123.763599,9.513984,13.020852,3.499305,0.077343,8.545512,2.804172,4157.68439,...,69.191944,21.978000,0,-1.296264,-1.154645,-0.530219,-0.681313,0.461862,1.951122,41.655059
613,0.435846,5462.03438,85.200147,46.551007,15.973224,5.979825,0.025882,12.622906,3.777550,5654.07556,...,124.808872,0.145340,0,1.064405,1.496189,0.800741,-3.426253,0.716409,4.043063,40.885888
614,0.427300,2459.10720,130.138587,55.355778,10.005552,8.070549,0.025578,15.408390,1.229900,5888.87769,...,119.559420,21.978000,0,0.454709,-0.162208,-0.720703,-3.770616,2.160889,4.445105,40.782968
615,0.363205,1263.53524,85.200147,23.685856,8.138688,7.981959,0.025578,7.524588,1.229900,4517.86560,...,37.155112,0.184622,0,-0.892913,-0.740511,0.179950,0.247562,0.014819,1.155425,41.399704


# t-SNE

In [55]:
tsne = TSNE(n_components = 2, random_state = 42)
tsne_df = pd.DataFrame(tsne.fit_transform(processed_df),
                      columns = ["tsne_Component1","tsne_Component2"])

In [56]:
kmeans.fit(tsne_df)
features_clusters = kmeans.predict(tsne_df)
tsne_distances = kmeans.transform(tsne_df)
tsne_distances_df = pd.DataFrame(tsne_distances, columns = ["tsne_distance1","tsne_distance2"])
tsne_df["tsne_Cluster"] = features_clusters
tsne_df = pd.concat([tsne_df,tsne_distances_df],axis =1)

C:\Users\ghibl\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

C:\Users\ghibl\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



In [57]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = tsne_df["tsne_Component1"], y = tsne_df["tsne_Component2"],mode = "markers",
                        marker = dict(color = tsne_df["tsne_Cluster"])))

In [58]:
tsne2 = TSNE(n_components = 3, random_state = 42)
tsne_df2 = pd.DataFrame(tsne2.fit_transform(processed_df),
                      columns = ["tsne_Component1","tsne_Component2","tsne_Component3"])

In [59]:
kmeans.fit(tsne_df2)
features_clusters2 = kmeans.predict(tsne_df2)
tsne_df2["tsne_Cluster"] = features_clusters

C:\Users\ghibl\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

C:\Users\ghibl\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



In [60]:
fig = go.Figure()
fig.add_trace(go.Scatter3d(x = tsne_df2["tsne_Component1"], y = tsne_df2["tsne_Component2"],z = tsne_df2["tsne_Component3"],
                         mode = "markers",marker = dict(color = tsne_df2["tsne_Cluster"])))

In [61]:
features_df = pd.concat([features_df,tsne_df.drop("tsne_Cluster", axis = 1)],axis = 1)

In [62]:
features_df.head()

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,pca_Component2,pca_Component3,pca_Component4,pca_Component5,pca_distance1,pca_distance2,tsne_Component1,tsne_Component2,tsne_distance1,tsne_distance2
0,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,...,-1.109691,1.198878,0.864153,0.202589,2.496555,41.978253,9.344210,-8.913967,12.277658,24.441946
1,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,...,-1.360409,-0.898552,-0.314330,0.676124,2.262222,41.708466,-15.278951,-3.763439,25.385233,1.173498
2,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,5135.78024,...,0.434018,0.720496,1.319426,-1.221434,1.985197,41.057363,0.523973,8.934161,10.216133,20.549118
3,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,4169.67738,...,0.262762,0.684072,0.890236,-1.520192,1.917203,41.388206,-5.519585,20.117620,22.228573,26.583408
4,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,5728.73412,...,0.788133,0.094595,1.569662,0.172711,1.793043,41.716341,18.158960,-9.161399,15.463756,33.196178


UMAP

In [63]:
um = UMAP(n_components = 2,random_state = 42)
um_df = pd.DataFrame(um.fit_transform(processed_df),
                     columns = ["um_Component1","um_Component2"])

In [64]:
kmeans.fit(um_df)
features_clusters = kmeans.predict(um_df)
um_distances = kmeans.transform(um_df)
um_distances_df = pd.DataFrame(um_distances, columns = ["um_distance1","um_distance2"])
um_df["um_Cluster"] = features_clusters
um_df = pd.concat([um_df,um_distances_df],axis =1)

C:\Users\ghibl\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

C:\Users\ghibl\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



In [65]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = um_df["um_Component1"], y = um_df["um_Component2"],mode = "markers",
                        marker = dict(color = um_df["um_Cluster"])))

In [66]:
um2 = UMAP(n_components = 3,random_state = 42)
um_df2 = pd.DataFrame(um2.fit_transform(processed_df),
                     columns = ["um_Component1","um_Component2","um_Component3"])

In [67]:
kmeans.fit(um_df2)
features_clusters = kmeans.predict(um_df2)
um_distances2 = kmeans.transform(um_df2)
um_distances_df2 = pd.DataFrame(um_distances2, columns = ["um_distance1","um_distance2"])
um_df2["um_Cluster"] = features_clusters
um_df2 = pd.concat([um_df2,um_distances_df2],axis =1)

C:\Users\ghibl\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

C:\Users\ghibl\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



In [68]:
fig = go.Figure()
fig.add_trace(go.Scatter3d(x = um_df2["um_Component1"], y = um_df2["um_Component2"],z = um_df2["um_Component3"],
                         mode = "markers",marker = dict(color = um_df2["um_Cluster"])))

In [69]:
features_df = pd.concat([features_df,um_df.drop("um_Cluster", axis = 1)],axis = 1)

In [70]:
features_df["Class"]

0      1
1      0
2      0
3      0
4      1
      ..
612    0
613    0
614    0
615    0
616    0
Name: Class, Length: 617, dtype: int64

Create columns to store the added and multiplied values for each feature combination

In [71]:
X_df.drop("Class",axis = 1,inplace = True)
columns = list(X_df.columns)
com_columns = list(itertools.combinations(columns, 2))
for com_col in com_columns:
    features_df[f"{com_col[0]}_{com_col[1]}_mul"] = complemented_df[f"{com_col[0]}"] *  complemented_df[f"{com_col[1]}"]
    features_df[f"{com_col[0]}_{com_col[1]}_sum"] = complemented_df[f"{com_col[0]}"] +  complemented_df[f"{com_col[1]}"]

C:\Users\ghibl\AppData\Local\Temp\ipykernel_19172\3071667330.py:6: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ghibl\AppData\Local\Temp\ipykernel_19172\3071667330.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ghibl\AppData\Local\Temp\ipykernel_19172\3071667330.py:6: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented 

C:\Users\ghibl\AppData\Local\Temp\ipykernel_19172\3071667330.py:6: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ghibl\AppData\Local\Temp\ipykernel_19172\3071667330.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ghibl\AppData\Local\Temp\ipykernel_19172\3071667330.py:6: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented 

C:\Users\ghibl\AppData\Local\Temp\ipykernel_19172\3071667330.py:6: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ghibl\AppData\Local\Temp\ipykernel_19172\3071667330.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ghibl\AppData\Local\Temp\ipykernel_19172\3071667330.py:6: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented 

C:\Users\ghibl\AppData\Local\Temp\ipykernel_19172\3071667330.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ghibl\AppData\Local\Temp\ipykernel_19172\3071667330.py:6: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ghibl\AppData\Local\Temp\ipykernel_19172\3071667330.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented 

C:\Users\ghibl\AppData\Local\Temp\ipykernel_19172\3071667330.py:6: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ghibl\AppData\Local\Temp\ipykernel_19172\3071667330.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ghibl\AppData\Local\Temp\ipykernel_19172\3071667330.py:6: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented 

C:\Users\ghibl\AppData\Local\Temp\ipykernel_19172\3071667330.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ghibl\AppData\Local\Temp\ipykernel_19172\3071667330.py:6: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ghibl\AppData\Local\Temp\ipykernel_19172\3071667330.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented 

C:\Users\ghibl\AppData\Local\Temp\ipykernel_19172\3071667330.py:6: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ghibl\AppData\Local\Temp\ipykernel_19172\3071667330.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ghibl\AppData\Local\Temp\ipykernel_19172\3071667330.py:6: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented 

C:\Users\ghibl\AppData\Local\Temp\ipykernel_19172\3071667330.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ghibl\AppData\Local\Temp\ipykernel_19172\3071667330.py:6: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ghibl\AppData\Local\Temp\ipykernel_19172\3071667330.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented 

C:\Users\ghibl\AppData\Local\Temp\ipykernel_19172\3071667330.py:6: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ghibl\AppData\Local\Temp\ipykernel_19172\3071667330.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ghibl\AppData\Local\Temp\ipykernel_19172\3071667330.py:6: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented 

In [72]:
features_df.drop("Class",axis = 1,inplace = True)
features_df_csv = pd.concat([dataset_df["Id"],features_df],axis = 1)
features_df_csv = pd.concat([features_df_csv, dataset_df["Class"]],axis = 1)
features_df_csv.to_csv(r'C:\Users\ghibl\ICR\data\input\all_features.csv',index = False)